In [1]:
import autograd

In [60]:
from __future__ import absolute_import
from __future__ import print_function
import matplotlib.pyplot as plt

import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy.stats.multivariate_normal as mvn
import autograd.scipy.stats.norm as norm
from autograd.scipy.special import expit


from autograd import grad
from autograd.optimizers import adam

In [61]:
# Generate data
mu1 = np.asarray([0.0, 0.0])
mu2 = np.asarray([1.0, 1.0])
sigma1 = np.asarray([0.5, 0.5])
sigma2 = np.asarray([0.5, 0.5])

M = 100
X1 = npr.randn(M / 2, 2) * sigma1 + mu1
X2 = npr.randn(M / 2, 2) * sigma2 + mu2
Y1 = np.zeros(M / 2)
Y2 = np.ones(M / 2)
X = np.concatenate((X1, X2), axis=0)
Y = np.concatenate((Y1, Y2), axis=0).reshape((M, 1))
XX = np.concatenate((X, Y), axis=1)
print(XX.shape)

(100, 3)


In [62]:
# Specify an inference problem by its unnormalized log-density.
D = 2
def log_density(Z, XX):
    """
    Z.shape = (M, D)
    X.shape = (N, D)
    """
    X = XX[:,:-1] # features
    Y = XX[:, -1] # labels
    # XX - design matrix N x M
    #print(X.shape, Z.shape)
    #P = expit(X.dot(Z.T))
    P = np.exp(-np.logaddexp(0, -X.dot(Z.T)))
    LL = Y * np.log(P) + (1 - Y) * np.log(1 - P)
    loss = LL.sum(axis=0).T
    return loss


def black_box_variational_inference(logprob, D, num_samples, XX):
    """Implements http://arxiv.org/abs/1401.0118, and uses the
    local reparameterization trick from http://arxiv.org/abs/1506.02557"""

    def unpack_latent_params(params):
        # Variational dist is a diagonal Gaussian.
        mean, log_std = params[:D], params[D:]
        return mean, log_std

    def gaussian_entropy(log_std):
        return 0.5 * D * (1.0 + np.log(2*np.pi)) + np.sum(log_std)

    rs = npr.RandomState(0)
    def variational_objective(latent_params, X):
        """Provides a stochastic estimate of the variational lower bound."""
        mean, log_std = unpack_latent_params(latent_params)
        samples = rs.randn(num_samples, D) * np.exp(log_std) + mean
        lower_bound = gaussian_entropy(log_std) + np.mean(logprob(samples, XX))
        return -lower_bound

    gradient = grad(variational_objective)

    return variational_objective, gradient, unpack_latent_params



In [63]:
%matplotlib inline
from IPython import display

def plot_isocontours(func, xlimits=[-2, 2], ylimits=[-4, 2], numticks=101):
     x = np.linspace(*xlimits, num=numticks)
     y = np.linspace(*ylimits, num=numticks)
     X, Y = np.meshgrid(x, y)
     zs = func(np.concatenate([np.atleast_2d(X.ravel()), np.atleast_2d(Y.ravel())]).T)
     Z = zs.reshape(X.shape)
     plt.contour(X, Y, Z)

def callback(params, t, g):
    print("Iteration {} lower bound {}".format(t, -objective(params, t)))
    plt.clf()

    #target_distribution = lambda x : np.exp(log_density(x, t))
    #plot_isocontours(target_distribution)

    mean, log_std = unpack_params(params)
    variational_contour = lambda x: mvn.pdf(x, mean, np.diag(np.exp(2*log_std)))
    plot_isocontours(variational_contour)
    plt.pause(1.0/30.0)

    display.clear_output(wait=True)
    display.display(plt.gcf())


In [64]:
objective, gradient, unpack_params = black_box_variational_inference(log_density, D, 200, XX)
init_mean    = -1 * np.ones(D)
init_log_std = -5 * np.ones(D)
init_var_params = np.concatenate([init_mean, init_log_std])

In [65]:
variational_params = adam(gradient, init_var_params, step_size=0.1, num_iters=100, callback=callback)

TypeError: ufunc 'logaddexp' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''